In [1]:
EXPT_NAME="Exp04: Kettle Reboiler"
import numpy as np
import sympy as sp
import pandas as pd

import platform
from datetime import datetime
print("**************************************")
print(EXPT_NAME)
print("Roll No: 120CH0057, Name: Siddharth Gautam")
print("Running On: Python ", platform.python_version(), " on: ", str(datetime.now()))
print("**************************************")
print("")

# Steam pressure
STEAM_PRESSURE = 1.7
# BUTANE properties
BUTANE_MFR = 10000 / 3600
BUTANE_INPUT_TEMPERATURE = 5 
BUTANE_PRESSURE = 5
BUTANE_HVAP = 386
BUTANE_MOL_WT = 58.12
# Critical Pressure (in bar)
BUTANE_P_CRITICAL = 37.9
# Density of liquid and vapor states
BUTANE_RHO = 604
# Heat capacity
BUTANE_CP = 1675
# Viscosity (cp)
BUTANE_MU = 0.007
# Antoine Equation Data
BUTANE_ANTOINE_A = 4.35776
BUTANE_ANTOINE_B = 1182.781
BUTANE_ANTOINE_C = 2.071
# U assumption
U_ASSUME = 1000
U_CLEAN = 10000
U_STEAM = 8000
kw_CARBON_STEEL = 55
U_FOULING = 5000
# Pipes
ID = 25 * (10**(-3))
OD = 30 * (10**(-3))
L = 4.8

T = sp.symbols('T', real=True, positive=True)
result = sp.solve([BUTANE_ANTOINE_A - (BUTANE_ANTOINE_B / (T - BUTANE_ANTOINE_C)) - np.log10(BUTANE_PRESSURE)], dict=True)
BUTANE_BOILING_PT = result[0][T] - 273.15
print(f"Calculated boiling point of BUTANE at {BUTANE_PRESSURE} bar pressure: {BUTANE_BOILING_PT}")

# Reboiler duty
sensible_heat = BUTANE_CP * (BUTANE_BOILING_PT - BUTANE_INPUT_TEMPERATURE)
total_heat_per_kg = BUTANE_HVAP + sensible_heat/1000
total_heat_per_kg *= 1.1
reboiler_duty = total_heat_per_kg * BUTANE_MFR
print(f"Reboiler Duty: {reboiler_duty} kW")

# Find steam saturation temperature
steam_table = pd.read_csv('steam-table.csv')
steam_pressure = steam_table['pressure']
steam_temperature = steam_table['temperature']
input_steam_temperature = np.interp(STEAM_PRESSURE, steam_pressure, steam_temperature) + 0.5
print(f"Input Steam Temperature: {input_steam_temperature} C")

# Calculate temperature difference and area
dT = input_steam_temperature - BUTANE_BOILING_PT
print(f"dT: {dT} C")
A = reboiler_duty * 1000 / (U_ASSUME * dT)
print(f"Area: {A} m^2")

# Calculate number of holes, round off to nearest even
N_holes = int(A / (np.pi * OD * L))
N_holes = N_holes + N_holes % 2
print(f"Number of holes: {N_holes}")

# Calculate pitch
PITCH = 1.5 * OD
print(f"Pitch: {PITCH*1000} mm")

# Calculate boiling coefficient now
q = reboiler_duty / A
print(f"Heat flux: {q} kW/m^2")

# Mostinski's equation
P_by_Pc = (BUTANE_PRESSURE/BUTANE_P_CRITICAL)
q_cb = 0.104*(BUTANE_P_CRITICAL**0.69)*((q*1000)**0.7)*(1.8*(P_by_Pc**0.17) + 4*(P_by_Pc**1.2) + 10*(P_by_Pc**10))
print(f"q_cb: {q_cb} kW/m^2")

# Calculation of U_0
U_0 = 1 / ((1/q_cb) + (1/U_CLEAN) + (OD*np.log(OD/ID)/(2*kw_CARBON_STEEL)) + (OD/ID)*((1/U_FOULING) + (1/U_STEAM)))
print(f"U_0: {U_0} W/m^2 K")

# Calculation of surface tension using Mac-Leod Sudgen
BUTANE_PARACHOR = 199.7
BUTANE_RHO_VAPOUR = (BUTANE_MOL_WT/22.4)*(273/(273 + BUTANE_BOILING_PT))*BUTANE_PRESSURE 
print(f"Density of BUTANE vapor: {BUTANE_RHO_VAPOUR} kg/m^3")
surface_tension = (BUTANE_PARACHOR * (BUTANE_RHO - BUTANE_RHO_VAPOUR)/(1000 * BUTANE_MOL_WT)) ** 4
surface_tension *= (1/1000) # convert to SI
print(f"Surface tension of butane: {surface_tension}")

# Calculation of maximum allowable heat flux
q_cb = 0.001*(0.44)*(PITCH/OD)*((BUTANE_HVAP*1000)/np.sqrt(N_holes/2))*(surface_tension*9.81*(BUTANE_RHO - BUTANE_RHO_VAPOUR)*(BUTANE_RHO_VAPOUR)**2)**0.25
ALLOWANCE_FACTOR = 0.7
q_cb *= ALLOWANCE_FACTOR
print(f"Maximum allowable heat flux: {q_cb} kW/m^2 K")

if(q < q_cb):
    print("*** SUCCESS: Design successful. Allowable heat flux is greater than calculated. ***")
else:
    print("*** ERROR: Design failed. Heat flux crosses allowable parameters. Retry with different parameters. ***")
    exit(-1)

# Vapor loading
VL = (2290 * (BUTANE_RHO_VAPOUR) * (surface_tension / (BUTANE_RHO - BUTANE_RHO_VAPOUR))**0.5)                                     /  3.185
print(f"Vapor loading: {VL}")
# Surface area
SA = (BUTANE_MFR/(L*VL))
print(f"Surace area: {SA} m^2")

# Shell Diamter (17 inches)
D_b = 17.25 * 0.0254
print(f"Bottom Diameter: {D_b} m")
# Calculated by hand
hD_s = 0.13
D_s = D_b/(1-hD_s)
print(f"Shell Diameter: {D_s} m")

result = { "Reboiler Duty (kW)": reboiler_duty, "Heat Flux (kW/m^2)": q, "Maximum Allowable Heat Flux": q_cb, "Number of holes": N_holes, "Outer Diameter (mm)": OD*1000, "Inner Diameter (mm)": ID*1000, "Pitch (mm)": PITCH*1000, "D_b (mm)": D_b*1000, "D_s (mm)": D_s*1000 }
df = pd.DataFrame(result.items(), columns=['Parameter', 'Value'])
display(df)

**************************************
Exp04: Kettle Reboiler
Roll No: 120CH0057, Name: Siddharth Gautam
Running On: Python  3.11.1  on:  2023-04-25 15:32:09.799628
**************************************

Calculated boiling point of BUTANE at 5 bar pressure: 52.1920818062010
Reboiler Duty: 1420.97614091090 kW
Input Steam Temperature: 115.44639175257733 C
dT: 63.2543099463764 C
Area: 22.4644951800997 m^2
Number of holes: 50
Pitch: 45.0 mm
Heat flux: 63.2543099463764 kW/m^2
q_cb: 4770.87217723045 kW/m^2
U_0: 1334.52668266745 W/m^2 K
Density of BUTANE vapor: 10.8910631535939 kg/m^3
Surface tension of butane: 0.0172483560585115
Maximum allowable heat flux: 372.547977561911 kW/m^2 K
*** SUCCESS: Design successful. Allowable heat flux is greater than calculated. ***
Vapor loading: 42.2282362363092
Surace area: 0.0137041883649906 m^2
Bottom Diameter: 0.43815 m
Shell Diameter: 0.5036206896551724 m


,Parameter,Value
0,Reboiler Duty (kW),1420.97614091090
1,Heat Flux (kW/m^2),63.2543099463764
2,Maximum Allowable Heat Flux,372.547977561911
3,Number of holes,50
4,Outer Diameter (mm),30.0
5,Inner Diameter (mm),25.0
6,Pitch (mm),45.0
7,D_b (mm),438.15
8,D_s (mm),503.62069
